# 简介

[Pydantic](https://docs.pydantic.dev/latest/)用于模式验证,类似`npm`的`zod`库.


# 示例

## 基本使用

In [1]:
import json
from datetime import datetime

import yaml
from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'tastes': {
        'wine': 9,
        b'cheese': 7,
        'cabbage': '1',
    },
}

user = User(**external_data)

print(user.id)
#> 123
print(user.model_dump())
"""
{
    'id': 123,
    'name': 'John Doe',
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},
}
"""

ModuleNotFoundError: No module named 'pydantic'

## 验证失败

验证失败时会抛出`ValidationError`异常,可以通过`e.errors()`获取错误信息.

In [ ]:
# continuing the above example...

from pydantic import ValidationError
import json


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}

try:
    User(**external_data)
except ValidationError as e:
    print(json.dumps(e.errors(), indent=4))
    """
    [
        {
            'type': 'int_parsing',
            'loc': ('id',),
            'msg': 'Input should be a valid integer, unable to parse string as an integer',
            'input': 'not an int',
            'url': 'https://errors.pydantic.dev/2/v/int_parsing',
        },
        {
            'type': 'missing',
            'loc': ('signup_ts',),
            'msg': 'Field required',
            'input': {'id': 'not an int', 'tastes': {}},
            'url': 'https://errors.pydantic.dev/2/v/missing',
        },
    ]
    """

## 数据类模板

In [ ]:
from pydantic import BaseModel, constr


class ClassName(BaseModel):
    int_no_default: int
    """
    整数,无默认值
    """
    int_with_default: int = 42
    """
    整数,有默认值
    """
    str_no_default: str
    """
    字符串,无默认值
    """
    str_with_default: str = 'foobar'
    """
    字符串,有默认值
    """
    str_with_length_limit: constr(min_length=3, max_length=10)
    """
    字符串,有长度限制
    """

## 将`json`转换为`pydantic`类

In [ ]:
from pydantic import BaseModel
import json

json_string = '''
{
    "int_no_default": "1",
    "str_no_default": "hello"
}
'''

# 将 JSON 字符串解析为字典
json_data = json.loads(json_string)

# 将字典转换为 Pydantic 类实例
class_instance = ClassName(**json_data)

print(class_instance)

## 将`yaml`转换为`pydantic`类

In [ ]:
from pydantic import BaseModel
import yaml


class OpenAIConfig(BaseModel):
    base_url: str
    """OpenAI 服务的基础 URL"""

    token: str
    """OpenAI 服务的访问令牌"""


class KidEngStoryConfig(BaseModel):
    openai: OpenAIConfig
    """OpenAI 配置"""


def load_config(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)


config = load_config(r'D:\Workspace\aiworld\py_automation\kid_eng_story\app.yml')

class_instance = KidEngStoryConfig(**config)

print(class_instance)

### 将`yaml`转换为泛型`pydantic`类

In [ ]:
from typing import TypeVar, Type
from pydantic import BaseModel

# 定义泛型类型变量
T = TypeVar('T', bound=BaseModel)


def load_yaml_as_pydantic(file_path: str, model: Type[T]) -> T:
    """
    读取 YAML 文件并将其转换为 Pydantic 类实例.

    :param file_path: YAML 文件路径
    :param model: Pydantic 泛型类型
    :return: Pydantic 类实例
    """
    with open(file_path, 'r') as file:
        yaml_data = yaml.safe_load(file)

    # 将 YAML 数据转换为 Pydantic 类实例
    return model(**yaml_data)


config = load_yaml_as_pydantic(r'D:\Workspace\aiworld\py_automation\kid_eng_story\app.yml', KidEngStoryConfig)
print(config)

## 生成`json schema`

最常用的场景是让`openai`返回符合`pydantic`类的`json schema`的数据.

In [ ]:
print(KidEngStoryConfig.model_json_schema())

# 使用`pydantic`管理配置文件

`python`里面一般用`.env`文件来存储配置信息,然后通过`os.getenv`或者`python-dotenv`来读取配置信息.但是这种方式有几个问题:

1. 需要手动转换类型,比如`os.getenv('A_BOOL')`返回的是`str`,需要手动转换为`bool`.
2. 不能忽略大小写,比如`os.getenv('A_BOOL')`和`os.getenv('a_bool')`是不同的.

`pydantic`可以解决这两个问题,通过继承`BaseSettings`类来创建自己的配置类,由`pydantic`来处理类型转换和大小写问题.

## 定义配置类

要使用`pydantic`管理配置文件,首先要定义一个配置类,继承`BaseSettings`类,然后定义配置项.

In [2]:
from pydantic_settings  import BaseSettings

class AppConfig(BaseSettings):
    react_version: str
    host: str
    port: int

## 从环境变量中读取配置

In [3]:
import os


os.environ['REACT_VERSION'] = '17.0.2'
os.environ['HOST'] = 'localhost'
os.environ['PORT'] = '3000'

config = AppConfig()
print(config.react_version)
#> 17.0.2
print(config.host)
#> localhost
print(config.port)
#> 3000

17.0.2
localhost
3000


## 带默认值的配置项

下面的例子中,`host`的默认值是`localhost`,如果环境变量中没有定义`host`,则使用默认值.

In [6]:
from pydantic_settings import BaseSettings

class AppConfig(BaseSettings):
    react_version: str
    host: str = 'localhost'
    port: int = 3000

config = AppConfig()
print(config.react_version)
#> 17.0.2
print(config.host)
#> localhost
print(config.port)

# 环境变量中的值将覆盖默认值
os.environ['HOST'] = '192.168.1.1'
config = AppConfig()
print(config.host)
#>  192.168.1.1
#> 3000

17.0.2
192.168.1.1
3000
192.168.1.1
